# Logistic Regression

In [1]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

In [2]:
# Read in the csv file
titanic_df = pd.read_csv(Path("data_files/titanic_cleaned.csv"))
titanic_df.head() # Review the dataframe

,passenger_id,pclass,survived,name,sex,age,sibsp,parch,fare,cabin,embarked,age_updated,embarked_updated,fare_updated,deck,pclass_updated
0,1,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,211.3375,B5,S,29.0000,Southampton (UK),211.3375,B,First class
1,2,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,151.5500,C22 C26,S,0.9167,Southampton (UK),151.5500,C,First class
2,3,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,151.5500,C22 C26,S,2.0000,Southampton (UK),151.5500,C,First class
3,4,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,151.5500,C22 C26,S,30.0000,Southampton (UK),151.5500,C,First class
4,5,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,151.5500,C22 C26,S,25.0000,Southampton (UK),151.5500,C,First class


In [3]:
# Create a copy of the dataframe
titanic = titanic_df.copy()

## Prepare the data

In [4]:
# Dropping columns not required as they have been updated, along with the 'name' column
titanic = titanic.drop(['passenger_id', 'pclass', 'name', 'age', 'embarked', 'fare', 'cabin'], axis=1)

titanic.head()

,survived,sex,sibsp,parch,age_updated,embarked_updated,fare_updated,deck,pclass_updated
0,1,female,0,0,29.0000,Southampton (UK),211.3375,B,First class
1,1,male,1,2,0.9167,Southampton (UK),151.5500,C,First class
2,0,female,1,2,2.0000,Southampton (UK),151.5500,C,First class
3,0,male,1,2,30.0000,Southampton (UK),151.5500,C,First class
4,0,female,1,2,25.0000,Southampton (UK),151.5500,C,First class


### Convert categorical data

In [5]:
# converting categorical using get_dummies and dropping the first column so the importance isn't inflated
sex_dummies = pd.get_dummies(titanic['sex'], drop_first=True) 
embarked_dummies = pd.get_dummies(titanic['embarked_updated'], drop_first=True)
deck_dummies = pd.get_dummies(titanic['deck'], drop_first=True)
pclass_dummies = pd.get_dummies(titanic['pclass_updated'], drop_first=True)

In [6]:
# Concatenate the encoded dummies with the dataframe
df_titanic_transformed = pd.concat([titanic,sex_dummies, embarked_dummies, deck_dummies,  pclass_dummies], axis=1)
df_titanic_transformed = df_titanic_transformed.drop(['sex', 'embarked_updated', 'deck', 'pclass_updated'], axis=1) # dropping columns not required
df_titanic_transformed.head()

,survived,sibsp,parch,age_updated,fare_updated,Second class,Third class,male,Queenstown (IE),Southampton (UK),B,C,D,E,F,G,T,Unknown
0,1,0,0,29.0000,211.3375,0,0,0,0,1,1,0,0,0,0,0,0,0
1,1,1,2,0.9167,151.5500,0,0,1,0,1,0,1,0,0,0,0,0,0
2,0,1,2,2.0000,151.5500,0,0,0,0,1,0,1,0,0,0,0,0,0
3,0,1,2,30.0000,151.5500,0,0,1,0,1,0,1,0,0,0,0,0,0
4,0,1,2,25.0000,151.5500,0,0,0,0,1,0,1,0,0,0,0,0,0


### Separate the data into labels and features

In [7]:
# Separate the features (X) from the target (y)
y = df_titanic_transformed["survived"]
X = df_titanic_transformed.drop(columns='survived')

In [8]:
# Reivew the y variable series
y.head()

0    1
1    1
2    0
3    0
4    0
Name: survived, dtype: int64

In [9]:
# Review the X variable dataframe
X.head()

,sibsp,parch,age_updated,fare_updated,Second class,Third class,male,Queenstown (IE),Southampton (UK),B,C,D,E,F,G,T,Unknown
0,0,0,29.0000,211.3375,0,0,0,0,1,1,0,0,0,0,0,0,0
1,1,2,0.9167,151.5500,0,0,1,0,1,0,1,0,0,0,0,0,0
2,1,2,2.0000,151.5500,0,0,0,0,1,0,1,0,0,0,0,0,0
3,1,2,30.0000,151.5500,0,0,1,0,1,0,1,0,0,0,0,0,0
4,1,2,25.0000,151.5500,0,0,0,0,1,0,1,0,0,0,0,0,0


In [10]:
# Check the balance of our target values
y.value_counts()

0    809
1    500
Name: survived, dtype: int64

## Split data into training and testing datasets using train_test_split

In [11]:
# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape


(981, 17)

## Logisitic regression model

In [30]:
# Create a logistic regression model
classifier = LogisticRegression(solver='lbfgs', max_iter=500, random_state=1)

# Fit and train the model using the training data
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=500, random_state=1)

In [13]:
# Make predictions
predictions = classifier.predict(X_test)
lr1_predictions_actuals_df = pd.DataFrame({"Prediction": predictions, "Actual": y_test})

## Evaluate the models performance

In [14]:
# Print the balanced_accuracy score of the model
print(f"The balanced accuracy score of the model is {balanced_accuracy_score(y_test, predictions)}")

The balanced accuracy score of the model is 0.7707389162561576


In [15]:
# Confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=['Actual - perished', 'Acutal - survived'], columns=['Predicted - perished', 'Predicted - survived']
)
print(cm)
display(cm_df)

[[183  20]
 [ 45  80]]


,Predicted - perished,Predicted - survived
Actual - perished,183,20
Acutal - survived,45,80


In [16]:
# Classification report
target_names = ["Perished", "Survived"]
print(classification_report(y_test, predictions, target_names=target_names))

              precision    recall  f1-score   support

    Perished       0.80      0.90      0.85       203
    Survived       0.80      0.64      0.71       125

    accuracy                           0.80       328
   macro avg       0.80      0.77      0.78       328
weighted avg       0.80      0.80      0.80       328



The model has a balanced accuracy score of 76% which is a result of the low recall score for predicting survivors.

Precision:
when the model predicts a survivor it correctly does so 79% of the time.  Indicating there are some survivors incorrectly being classified as not having survived.
When the model predicts someone who perished, it correctly do so 80% of the time.

Recall
This looks at when the passenger actually survived, how often the model correcly predicts them as surviving.  This model will classify a survivor correctly just 63% of the time.
When a passenger actually perished, the model will classify them correctly as perishing 90% of the time.

the model seems slightly more accurate at predicting those who perished than those who survived.

# Scaling the data and re-running model

In [17]:
# creating standardscaler instance
scaler = StandardScaler() 

# fitting the standard scaler
X_scaler = scaler.fit(X_train) 

# scaling the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Logistic regression model

In [18]:
# Create a logistic regression model
classifier = LogisticRegression(solver='lbfgs', max_iter=100, random_state=1)

# Fit and train the model using the training data
classifier.fit(X_train_scaled, y_train)

LogisticRegression(random_state=1)

In [19]:
# Make predictions
predictions = classifier.predict(X_test_scaled)
lr_one_scaled_predics_df = pd.DataFrame({"Prediction": predictions, "Actual": y_test})

### Evaluate the models performance

In [20]:
# Print the balanced_accuracy score of the model
print(f"The balanced accuracy score of the model is {balanced_accuracy_score(y_test, predictions)}")


# Confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=['Actual - perished', 'Acutal - survived'], columns=['Predicted - perished', 'Predicted - survived']
)
print(cm)
display(cm_df)


# Classification report
target_names = ["Perished", "Survived"]
print(classification_report(y_test, predictions, target_names=target_names))


The balanced accuracy score of the model is 0.7722758620689656
[[182  21]
 [ 44  81]]


,Predicted - perished,Predicted - survived
Actual - perished,182,21
Acutal - survived,44,81


              precision    recall  f1-score   support

    Perished       0.81      0.90      0.85       203
    Survived       0.79      0.65      0.71       125

    accuracy                           0.80       328
   macro avg       0.80      0.77      0.78       328
weighted avg       0.80      0.80      0.80       328



# Applying random over sampling due to the imbalanced data and re-running model

In [21]:
# Instantiate the random oversample model
ros = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_train_ros, y_train_ros= ros.fit_resample(X_train, y_train)

# Count the distinct values of the resampled labels data
y_train_ros.value_counts()

1    606
0    606
Name: survived, dtype: int64

### Create logistic regression model with the original data

In [22]:
# Create a logistic regression model
ros_model = LogisticRegression(solver='lbfgs', max_iter=600, random_state=1)

# Fit and train the model using the training data
ros_model.fit(X_train_ros, y_train_ros)

# Make predictions
ros_predictions = ros_model.predict(X_test)

### Evaluate the models performance

In [23]:
# Print the balanced_accuracy score of the model
print(f"The balanced accuracy score of the model is {balanced_accuracy_score(y_test, ros_predictions)}")

The balanced accuracy score of the model is 0.7796453201970444


In [24]:
# Confusion matrix
cm = confusion_matrix(y_test, ros_predictions)
cm_df = pd.DataFrame(
    cm, index=['Actual - perished', 'Acutal - survived'], columns=['Predicted - perished', 'Predicted - survived']
)
print(cm)
display(cm_df)

[[172  31]
 [ 36  89]]


,Predicted - perished,Predicted - survived
Actual - perished,172,31
Acutal - survived,36,89


In [25]:
# Classification report
target_names = ["Perished", "Survived"]
print(classification_report(y_test, ros_predictions, target_names=target_names))


              precision    recall  f1-score   support

    Perished       0.83      0.85      0.84       203
    Survived       0.74      0.71      0.73       125

    accuracy                           0.80       328
   macro avg       0.78      0.78      0.78       328
weighted avg       0.79      0.80      0.79       328

